# PyTorch feature extraction
- VGG
- ResNet 

In [1]:
import pretrainedmodels
# https://github.com/mratsim/Amazon-Forest-Computer-Vision/blob/master/src/p_neuro.py
# import tensorflow as tf
# global graph,model
# graph = tf.get_default_graph()
import torchvision.models as models
import torch 
import pretrainedmodels
from glob import glob
import os
import pickle
from PIL import Image
import numpy as np
from pathlib import Path

import glob
import os
import pickle
from PIL import Image
from tqdm import tqdm 
from glob import glob
import os
from glob import glob
from matplotlib.pyplot import imshow
import numpy as np
from PIL import Image
from pathlib import Path
import numpy as np
import pandas as pd
import collections

import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

import torch
import torchvision
from torchvision import transforms
import torchvision.models as models
from torch.utils.data.dataset import Dataset
from PIL import Image
from tqdm import tqdm 
import os
import sys
import PIL
from torch.autograd import Variable
torch.backends.cudnn.benchmark=True
from torch import nn
import math 
import h5py
import numpy

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True



In [3]:
imgnet_size=(224,224)

## Augmentation + Normalization for full training
TTA_trans = transforms.Compose([
    transforms.Resize(imgnet_size),
#     PowerPIL(),
    transforms.ToTensor(),    
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

faces_trans = transforms.Compose([
    transforms.Resize(imgnet_size), 
    transforms.ToTensor()]) # for faces only 

test_trans = transforms.Compose([
    transforms.Resize(imgnet_size),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])]) 

def flaotTensorToImage(img, mean=0, std=1):
        """convert a tensor to an image"""
        img = np.transpose(img.numpy(), (1, 2, 0))
        img = (img*std+ mean)*255
        img = img.astype(np.uint8)    
        return img    

class FeaturesImageFolder(Dataset):
    def __init__(self, root, transform=test_trans):
        images = []
        for filename in os.listdir(root):
            if filename.endswith('jpg') or filename.endswith('jpeg') or filename.endswith('png') or filename.endswith('gif'):
#             if filename.endswith('jpg'):                    
                images.append('{}'.format(filename))

        self.root = root
        self.imgs = images
        self.transform = transform

    def __getitem__(self, index):
        filename = self.imgs[index]
        img = Image.open(os.path.join(self.root, filename))
        img = img.convert('RGB')        
        img = self.transform(img)
            
        return img, filename

    def __len__(self):
        return len(self.imgs)



class ResNetBottom(torch.nn.Module):
    def __init__(self, original_model, real_name, num_feats):        
        super(ResNetBottom, self).__init__()
        self.features = torch.nn.Sequential(*list(original_model.children())[:-1])
        self.real_name = real_name        
        self.num_feats = num_feats
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        return x
    
class VGG16Bottom(torch.nn.Module):
        def __init__(self):
            super(VGG16Bottom, self).__init__()
            original_model = models.vgg16(pretrained=True)
            self.real_name = (((type(original_model).__name__)))
            self.real_name = "vgg19"
            
            self.features = original_model.features                        
            self.classifier = torch.nn.Sequential(*list(original_model.classifier.children())[:-1])            
            self.num_feats = 4096            

        def forward(self, x):
            f = self.features(x)
            f = f.view(f.size(0), -1) # (1, 4096) -> (4096, )
            f = self.classifier(f)
#             print (f.data.size())
            return f



def single_vector(image, model,real_name, TTA=False):     
    
    f=None    
    image = test_trans(image)                
    image = image.unsqueeze(0)  #this is for VGG, may not be needed for ResNet        
    image = Variable(image, requires_grad=False).cuda()        
    image= image.cuda()
    f = model(image) 
    
    if real_name.startswith("vgg19"):
        f = f.view(f.size(0), -1)
        print ("Size : {}".format(f.shape))    
    elif real_name.startswith("res34"):
        f = f.view(f.size(1), -1)
        print ("Size : {}".format(f.shape))    
        f = f.view(f.size(1),-1)     
        print ("Size : {}".format(f.shape))                
                
    f =f.cpu().detach().numpy()[0]
    print ("Size : {}".format(f.shape))        
    return f



def getModel(real_name):
    model=None    
    if real_name.startswith("vgg19"):
            vgg_model=VGG16Bottom()
            for param in vgg_model.parameters():
                param.requires_grad = False    
            vgg_model.cuda()
            vgg_model.eval()
            model=vgg_model                                
    elif real_name.startswith("res"):
        num_feats=None
        res_model=None
        if real_name.startswith("res34"):
            res_model = models.resnet34(pretrained=True)
            num_feats=512
        elif real_name.startswith("res152"):
            res_model = models.resnet152(pretrained=True)            
            num_feats=2048
        res_model.eval()                
        
        res_conv2 = ResNetBottom(res_model, real_name=real_name, num_feats=num_feats)
        for param in res_conv2.parameters():
            param.requires_grad = False        
        model=res_conv2.cuda()           
        res_conv2.eval()
        model=res_conv2                
    else:
        model=None
        print ("Model not supported") 
            
    real_name=(((type(model).__name__)))      
    model.cuda()    
    model = torch.nn.DataParallel(model, device_ids=range(torch.cuda.device_count()))
    print (model)
    
    return model 

In [4]:
input_folder='/root/img/'
input_dataset = FeaturesImageFolder(input_folder, transform=test_trans)
print ("Num images: {}".format(len(input_dataset)))
bs = 1
kwargs = {'num_workers': 8, 'pin_memory': True}
image_loader = torch.utils.data.DataLoader(input_dataset, batch_size=bs,**kwargs)

Num images: 88620


In [5]:
import torchvision.models as models
import torch 

real_name="vgg19"

model=getModel(real_name)
# model.cuda()
# model = torch.nn.DataParallel(model, device_ids=range(torch.cuda.device_count()))

DataParallel(
  (module): VGG16Bottom(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace)
      (16): MaxPool2d(kernel_size

In [6]:
import os
import os.path
print ("Model: {}".format ((type(model).__name__)))
print('Total params: %.2fM' % (sum(p.numel() for p in model.parameters()) / 1000000.0))            

if __name__ == '__main__':    
    with torch.no_grad():
        model.eval()
        for i, data in tqdm (enumerate(image_loader)):
            images, img_path = data  
            images.cuda()
            features = model(images) # a feature vecort, 4096 or 2048
            for i in  (range(0, len(features))):
                real_ext=os.path.splitext(os.path.basename(img_path[0]))[1]
                
                features_dir='static/feature/' + real_name 
                if not os.path.exists(features_dir):
                    os.makedirs(features_dir)
    
                feature_path = features_dir + "/" + os.path.splitext(os.path.basename(img_path[0]))[0] + real_ext +'.pkl'                
                f =features[i].cpu().detach().numpy()
                f= f / np.linalg.norm(f)
                pickle.dump(f, open(feature_path, 'wb'))                
                    

Model: DataParallel
Total params: 134.26M


759it [00:19, 105.30it/s]

KeyboardInterrupt: 

Exception in thread Thread-5:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 158, in _pin_memory_loop
    r = in_queue.get(timeout=MP_STATUS_CHECK_INTERVAL)
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 113, in get
    return _ForkingPickler.loads(res)
  File "/usr/local/lib/python3.6/dist-packages/torch/multiprocessing/reductions.py", line 256, in rebuild_storage_fd
    fd = df.detach()
  File "/usr/lib/python3.6/multiprocessing/resource_sharer.py", line 57, in detach
    with _resource_sharer.get_connection(self._id) as conn:
  File "/usr/lib/python3.6/multiprocessing/resource_sharer.py", line 87, in get_connection
    c = Client(address, authkey=process.current_process().authkey)
  File "/usr/lib/python3.6/mu